## Demonstration of the topic coherence pipeline in Gensim

### Introduction

We will be using the `u_mass` and `c_v` coherence for two different LDA models: a "good" and a "bad" LDA model. The good LDA model will be trained over 50 iterations and the bad one for 1 iteration. Hence in theory, the good LDA model will be able come up with better or more human-understandable topics. Therefore the coherence measure output for the good LDA model should be more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable.

In [1]:
import numpy as np
import logging
try:
    import pyLDAvis.gensim
except ImportError:
    ValueError("SKIP: please install pyLDAvis")
    
import json
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.hdpmodel import HdpModel
from gensim.models.wrappers import LdaVowpalWabbit, LdaMallet
from gensim.corpora.dictionary import Dictionary
from numpy import array

### Set up logging

In [2]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

### Set up corpus

As stated in table 2 from [this](http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf) paper, this corpus essentially has two classes of documents. First five are about human-computer interaction and the other four are about graphs. We will be setting up two LDA models. One with 50 iterations of training and the other with just 1. Hence the one with 50 iterations ("better" model) should be able to capture this underlying pattern of the corpus better than the "bad" LDA model. Therefore, in theory, our topic coherence for the good LDA model should be greater than the one for the bad LDA model.

In [3]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

In [4]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(12 unique tokens: ['graph', 'system', 'user', 'minors', 'human']...) from 9 documents (total 29 corpus positions)


### Set up two topic models

We'll be setting up two different LDA Topic models. A good one and bad one. To build a "good" topic model, we'll simply train it using more iterations than the bad one. Therefore the `u_mass` coherence should in theory be better for the good model than the bad one since it would be producing more "human-interpretable" topics.

In [5]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=2)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.5
INFO:gensim.models.ldamodel:using symmetric eta at 0.08333333333333333
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 2 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity and coherence every 9 documents, iterating 50x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-3.297 per-word bound, 9.8 perplexity estimate based on a held-out corpus of 9 documents with 29 words
INFO:gensim.models.ldamodel:-15.478 coherence estimate based on a held-out corpus of 9 documents with 29 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #9/9
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:6/9 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updat

In [6]:
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=2)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.5
INFO:gensim.models.ldamodel:using symmetric eta at 0.08333333333333333
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 2 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity and coherence every 9 documents, iterating 1x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-3.317 per-word bound, 10.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
INFO:gensim.models.ldamodel:-16.260 coherence estimate based on a held-out corpus of 9 documents with 29 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #9/9
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:0/9 documents converged within 1 iterations
DEBUG:gensim.models.ldamodel:updati

Now, the LdaModel in gensim also logs the evaluation parameters `perplexity` and `coherence` according to `eval_every`. These parameter values can be used to monitor the LDA training and evaluate how the topics are improving during training. 

### Using U_Mass Coherence

In [7]:
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [8]:
badcm = CoherenceModel(model=badLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

### View the pipeline parameters for one coherence model

Following are the pipeline parameters for `u_mass` coherence. By pipeline parameters, we mean the functions being used to calculate segmentation, probability estimation, confirmation measure and aggregation as shown in figure 1 in [this](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) paper.

In [16]:
print goodcm

CoherenceModel(segmentation=<function s_one_pre at 0x7f663ae82f50>, probability estimation=<function p_boolean_document at 0x7f663ae8a2a8>, confirmation measure=<function log_conditional_probability at 0x7f663ae8a668>, aggregation=<function arithmetic_mean at 0x7f663ae8aa28>)


### Interpreting the topics

As we will see below using LDA visualization, the better model comes up with two topics composed of the following words:
1. goodLdaModel:
    - __Topic 1__: More weightage assigned to words such as "system", "user", "eps", "interface" etc which captures the first set of documents.
    - __Topic 2__: More weightage assigned to words such as "graph", "trees", "survey" which captures the topic in the second set of documents.
2. badLdaModel:
    - __Topic 1__: More weightage assigned to words such as "system", "user", "trees", "graph" which doesn't make the topic clear enough.
    - __Topic 2__: More weightage assigned to words such as "system", "trees", "graph", "user" which is similar to the first topic. Hence both topics are not human-interpretable.

Therefore, the topic coherence for the goodLdaModel should be greater for this than the badLdaModel since the topics it comes up with are more human-interpretable. We will see this using `u_mass` and `c_v` topic coherence measures.

### Visualize topic models

In [17]:
pyLDAvis.enable_notebook()

In [18]:
pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics        x    y
topic                                          
1      60.467874        1       1 -0.02178 -0.0
0      39.532126        1       2  0.02178 -0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
1     Default  2.000000      graph  2.000000  12.0000  12.0000
6     Default  2.000000     survey  2.000000  11.0000  11.0000
3     Default  2.000000      trees  2.000000  10.0000  10.0000
0     Default  2.000000     minors  2.000000   9.0000   9.0000
5     Default  2.000000   computer  2.000000   8.0000   8.0000
4     Default  2.000000        eps  2.000000   7.0000   7.0000
9     Default  2.000000       time  2.000000   6.0000   6.0000
11    Default  2.000000   response  2.000000   5.0000   5.0000
2     Default  3.000000     system  3.000000   4.0000   4.0000
7     Default  2.000000       user  2.000000   3.0000   3.0000
8     Default  2.000000      human  2.000000   2.0000   2.0000
10    Default  2.000000  interface  2.000000   1.0000   1.0000
4      Topic1  1.754656        eps  2.192159   0.2804  -2.3020
2      Topic1  2.765990     system  3.630010   0.2312  -1.8468
7      Topic1  2.132646       user  2.892076   0.1984  -2.1069
10     Topic1  1.511120  interface  2.155900   0.1477  -2.4514
8      Topic1  1.448214      human  2.146535   0.1095  -2.4939
11     Topic1  1.300499   response  2.124542   0.0122  -2.6015
9      Topic1  1.292999       time  2.123425   0.0070  -2.6073
3      Topic1  1.420436      trees  2.786037  -0.1706  -2.5133
5      Topic1  1.064564   computer  2.089414  -0.1713  -2.8017
0      Topic1  1.037844     minors  2.085436  -0.1948  -2.8271
6      Topic1  0.818827     survey  2.052828  -0.4160  -3.0641
1      Topic1  0.987888      graph  2.721637  -0.5104  -2.8764
1      Topic2  1.733749      graph  2.721637   0.4771  -1.8890
6      Topic2  1.234000     survey  2.052828   0.4191  -2.2290
0      Topic2  1.047592     minors  2.085436   0.2396  -2.3927
5      Topic2  1.024850   computer  2.089414   0.2157  -2.4147
3      Topic2  1.365602      trees  2.786037   0.2150  -2.1276
9      Topic2  0.830426       time  2.123425  -0.0108  -2.6251
11     Topic2  0.824043   response  2.124542  -0.0190  -2.6328
8      Topic2  0.698320      human  2.146535  -0.1949  -2.7983
10     Topic2  0.644780  interface  2.155900  -0.2790  -2.8781
7      Topic2  0.759429       user  2.892076  -0.4091  -2.7144
2      Topic2  0.864020     system  3.630010  -0.5073  -2.5854
4      Topic2  0.437504        eps  2.192159  -0.6835  -3.2659, token_table=      Topic      Freq       Term
term                            
5         1  0.478603   computer
5         2  0.478603   computer
4         1  0.912342        eps
1         1  0.367426      graph
1         2  0.734852      graph
8         1  0.465867      human
8         2  0.465867      human
10        1  0.927687  interface
10        2  0.463843  interface
0         1  0.479516     minors
0         2  0.479516     minors
11        1  0.470690   response
11        2  0.470690   response
6         1  0.487133     survey
6         2  0.487133     survey
2         1  0.826444     system
2         2  0.275481     system
9         1  0.470937       time
9         2  0.470937       time
3         1  0.358933      trees
3         2  0.358933      trees
7         1  0.691545       user
7         2  0.345772       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [19]:
pyLDAvis.gensim.prepare(badLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
1      52.514671        1       1 -0.002455 -0.0
0      47.485329        1       2  0.002455 -0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
8     Default  2.000000      human  2.000000  12.0000  12.0000
4     Default  2.000000        eps  2.000000  11.0000  11.0000
1     Default  2.000000      graph  2.000000  10.0000  10.0000
9     Default  2.000000       time  2.000000   9.0000   9.0000
5     Default  2.000000   computer  2.000000   8.0000   8.0000
3     Default  2.000000      trees  2.000000   7.0000   7.0000
6     Default  2.000000     survey  2.000000   6.0000   6.0000
10    Default  2.000000  interface  2.000000   5.0000   5.0000
0     Default  2.000000     minors  2.000000   4.0000   4.0000
2     Default  3.000000     system  3.000000   3.0000   3.0000
7     Default  2.000000       user  2.000000   2.0000   2.0000
11    Default  2.000000   response  2.000000   1.0000   1.0000
9      Topic1  1.315907       time  2.123095   0.1657  -2.4487
6      Topic1  1.228044     survey  2.122596   0.0969  -2.5178
0      Topic1  1.189171     minors  2.122376   0.0648  -2.5500
11     Topic1  1.156021   response  2.122188   0.0366  -2.5782
2      Topic1  1.926266     system  3.536977   0.0364  -2.0676
7      Topic1  1.540934       user  2.829581   0.0363  -2.2908
10     Topic1  1.134199  interface  2.122064   0.0176  -2.5973
3      Topic1  1.477609      trees  2.829222  -0.0055  -2.3328
5      Topic1  1.032319   computer  2.121486  -0.0762  -2.6914
1      Topic1  1.347614      graph  2.828485  -0.0973  -2.4249
4      Topic1  0.977820        eps  2.121177  -0.1303  -2.7456
8      Topic1  0.903351      human  2.120755  -0.2093  -2.8249
8      Topic2  1.217404      human  2.120755   0.1897  -2.4258
4      Topic2  1.143357        eps  2.121177   0.1267  -2.4886
1      Topic2  1.480871      graph  2.828485   0.0976  -2.2299
5      Topic2  1.089167   computer  2.121486   0.0780  -2.5371
3      Topic2  1.351613      trees  2.829222   0.0060  -2.3212
10     Topic2  0.987865  interface  2.122064  -0.0198  -2.6348
7      Topic2  1.288647       user  2.829581  -0.0418  -2.3690
2      Topic2  1.610711     system  3.536977  -0.0418  -2.1459
11     Topic2  0.966167   response  2.122188  -0.0421  -2.6570
0      Topic2  0.933205     minors  2.122376  -0.0769  -2.6917
6      Topic2  0.894553     survey  2.122596  -0.1193  -2.7340
9      Topic2  0.807188       time  2.123095  -0.2223  -2.8367, token_table=      Topic      Freq       Term
term                            
5         1  0.471368   computer
5         2  0.471368   computer
4         1  0.471436        eps
4         2  0.471436        eps
1         1  0.353546      graph
1         2  0.353546      graph
8         1  0.471530      human
8         2  0.471530      human
10        1  0.471239  interface
10        2  0.471239  interface
0         1  0.471170     minors
0         2  0.471170     minors
11        1  0.471212   response
11        2  0.471212   response
6         1  0.471121     survey
6         2  0.471121     survey
2         1  0.565455     system
2         2  0.565455     system
9         1  0.471011       time
9         2  0.471011       time
3         1  0.353454      trees
3         2  0.353454      trees
7         1  0.706818       user
7         2  0.353409       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [20]:
print goodcm.get_coherence()

-14.0842451581


In [21]:
print badcm.get_coherence()

-14.4434307511


### Using C_V coherence

In [25]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')

In [26]:
badcm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')

### Pipeline parameters for C_V coherence

In [27]:
print goodcm

CoherenceModel(segmentation=<function s_one_set at 0x7f663ae8a050>, probability estimation=<function p_boolean_sliding_window at 0x7f663ae8a320>, confirmation measure=<function cosine_similarity at 0x7f663ae8a938>, aggregation=<function arithmetic_mean at 0x7f663ae8aa28>)


### Print coherence values

In [28]:
print goodcm.get_coherence()

0.552164532134


In [29]:
print badcm.get_coherence()

0.5269189184


### Support for wrappers

This API supports gensim's _ldavowpalwabbit_ and _ldamallet_ wrappers as input parameter to `model`.

In [5]:
model1 = LdaVowpalWabbit('/home/devashish/vw-8', corpus=corpus, num_topics=2, id2word=dictionary, passes=50)
model2 = LdaVowpalWabbit('/home/devashish/vw-8', corpus=corpus, num_topics=2, id2word=dictionary, passes=1)

In [7]:
cm1 = CoherenceModel(model=model1, corpus=corpus, coherence='u_mass')
cm2 = CoherenceModel(model=model2, corpus=corpus, coherence='u_mass')

In [8]:
print cm1.get_coherence()
print cm2.get_coherence()

-14.075813889
-15.1740896045


In [20]:
model1 = LdaMallet('/home/devashish/mallet-2.0.8RC3/bin/mallet',corpus=corpus , num_topics=2, id2word=dictionary, iterations=50)
model2 = LdaMallet('/home/devashish/mallet-2.0.8RC3/bin/mallet',corpus=corpus , num_topics=2, id2word=dictionary, iterations=1)

In [21]:
cm1 = CoherenceModel(model=model1, texts=texts, coherence='c_v')
cm2 = CoherenceModel(model=model2, texts=texts, coherence='c_v')

In [22]:
print cm1.get_coherence()
print cm2.get_coherence()

0.581114877802
0.549865328265


### Support for other topic models
The gensim topics coherence pipeline can be used with other topics models too. Only the tokenized `topics` should be made available for the pipeline. Eg. with the gensim HDP model

In [5]:
hm = HdpModel(corpus=corpus, id2word=dictionary)

In [8]:
# To get the topic words from the model
topics = []
for topic_id, topic in hm.show_topics(num_topics=10, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [9]:
topics[:2]

[[u'minors',
  u'system',
  u'graph',
  u'human',
  u'interface',
  u'eps',
  u'trees',
  u'computer',
  u'user',
  u'response',
  u'survey',
  u'time'],
 [u'minors',
  u'trees',
  u'time',
  u'interface',
  u'user',
  u'survey',
  u'system',
  u'response',
  u'human',
  u'computer',
  u'graph',
  u'eps']]

In [10]:
# Initialize CoherenceModel using `topics` parameter
cm = CoherenceModel(topics=topics, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [11]:
cm.get_coherence()

-14.640667699204982

### Conclusion

Hence as we can see, the `u_mass` and `c_v` coherence for the good LDA model is much more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable. The badLdaModel however fails to decipher between these two topics and comes up with topics which are not clear to a human. The `u_mass` and `c_v` topic coherences capture this wonderfully by giving the interpretability of these topics a number as we can see above. Hence this coherence measure can be used to compare difference topic models based on their human-interpretability.